In [3]:
# =========================================
# AI Agent: Fetch NSE Stocks and Analyze % Price Change (Dynamic Pages)
# Named Agent As "DynamicStock Agent - India"
# =========================================

# Import necessary libraries
import requests
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import concurrent.futures

# =========================================
# Step 1: Dynamically Fetch All Active NSE Stocks
# =========================================
import pandas as pd
import requests
from io import StringIO   # <-- ADD this!

def fetch_all_active_stocks():
    print("# Fetching all active NSE stocks...")

    try:
        # NSE India Listed Stocks
        nse_url = "https://archives.nseindia.com/content/equities/EQUITY_L.csv"
        nse_headers = {
            "User-Agent": "Mozilla/5.0"
        }

        nse_response = requests.get(nse_url, headers=nse_headers)

        if nse_response.status_code != 200:
            print(f"❌ Failed to fetch NSE stocks. Status code: {nse_response.status_code}")
            return []

        nse_content = nse_response.content.decode('utf-8')
        nse_df = pd.read_csv(StringIO(nse_content))   # <-- Fixed here!

        nse_df = nse_df[['SYMBOL', 'NAME OF COMPANY']]
        nse_df = nse_df.dropna()

        stocks_list = list(zip(nse_df['SYMBOL'], nse_df['NAME OF COMPANY']))

        print(f"✅ Total NSE stocks fetched: {len(stocks_list)}")
        return stocks_list

    except Exception as e:
        print(f"⚠️ Error fetching NSE stocks list: {e}")
        return []

# =========================================
# Step 2: Fetch Stock Prices (Current and 6 Months Ago)
# =========================================
def get_stock_prices(ticker):
    stock = yf.Ticker(ticker)

    try:
        current_price_data = stock.history(period="1d")
        current_price = current_price_data['Close'][-1]
        current_price_date = current_price_data.index[-1].date()
    except:
        current_price, current_price_date = None, None

    six_months_ago = datetime.now() - timedelta(days=182)
    try:
        historical_data = stock.history(start=six_months_ago, end=six_months_ago + timedelta(days=5))
        if not historical_data.empty:
            past_price = historical_data['Close'][0]
            past_price_date = historical_data.index[0].date()
        else:
            past_price, past_price_date = None, None
    except:
        past_price, past_price_date = None, None

    return current_price, current_price_date, past_price, past_price_date

# =========================================
# Step 3: Build the Stocks DataFrame (Multithreaded)
# =========================================
def process_single_stock(ticker, name):
    ticker = ticker + ".NS"  # Add NSE suffix
    current_price, current_date, past_price, past_date = get_stock_prices(ticker)

    if current_price and past_price:
        percent_change = ((current_price - past_price) / past_price) * 100
    else:
        percent_change = None

    return {
        "Stock Name": name,
        "Ticker": ticker,
        "Current Stock Price": current_price,
        "Current Price Date": current_date,
        "180 Days Back Stock Price": past_price,
        "180 Days Back Price Date": past_date,
        "% Change": percent_change
    }

def build_stocks_dataframe(stocks_list):
    print("# Building the main DataFrame...")

    data = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(process_single_stock, ticker, name) for ticker, name in stocks_list]
        for idx, future in enumerate(concurrent.futures.as_completed(futures)):
            try:
                result = future.result()
                data.append(result)
                if idx % 10 == 0:
                    print(f"--> Processed {idx + 1}/{len(stocks_list)} stocks")
            except Exception as e:
                print(f"⚠️ Error processing stock: {e}")

    df = pd.DataFrame(data)
    print("✅ DataFrame built successfully.")
    return df

# =========================================
# Step 4: Filter Stocks Based on % Change
# =========================================
def filter_stocks(df, filter_type="drop"):
    if filter_type == "drop":
        return df[df["% Change"] <= -50]
    elif filter_type == "rise":
        return df[df["% Change"] >= 50]
    else:
        return df

# =========================================
# Step 5: Export Results to CSV and Excel
# =========================================
def export_results(df, base_filename):
    today = datetime.today().strftime('%Y-%m-%d')
    csv_filename = f"{base_filename}_{today}.csv"
    excel_filename = f"{base_filename}_{today}.xlsx"

    df.to_csv(csv_filename, index=False)
    df.to_excel(excel_filename, index=False)

    print(f"✅ Results exported to '{csv_filename}' and '{excel_filename}' successfully.")

# =========================================
# Step 6: Main AI Agent Runner
# =========================================
def ai_agent_run():
    print("# Starting AI agent...")

    # Step 1: Fetch all NSE stocks
    stocks_list = fetch_all_active_stocks()

    # Optional: Limit during testing
    # stocks_list = stocks_list[:100]

    # Step 2 & 3: Build DataFrame
    df = build_stocks_dataframe(stocks_list)

    # Step 4: Export all data
    export_results(df, "nse_all_stocks_data")

    # Step 5: Filter and export filtered data
    df_rise = filter_stocks(df, "rise")
    df_drop = filter_stocks(df, "drop")

    export_results(df_rise, "nse_stocks_50_percent_rise")
    export_results(df_drop, "nse_stocks_50_percent_drop")

    print("# AI Agent execution completed successfully!")

# =========================================
# Entry Point
# =========================================
if __name__ == "__main__":
    ai_agent_run()


# Starting AI agent...
# Fetching all active NSE stocks...
✅ Total NSE stocks fetched: 2081
# Building the main DataFrame...


C:\Users\kchak\AppData\Local\Temp\ipykernel_9028\3222473881.py:59: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = current_price_data['Close'][-1]
C:\Users\kchak\AppData\Local\Temp\ipykernel_9028\3222473881.py:59: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = current_price_data['Close'][-1]
C:\Users\kchak\AppData\Local\Temp\ipykernel_9028\3222473881.py:59: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  c

--> Processed 1/2081 stocks


C:\Users\kchak\AppData\Local\Temp\ipykernel_9028\3222473881.py:68: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  past_price = historical_data['Close'][0]
C:\Users\kchak\AppData\Local\Temp\ipykernel_9028\3222473881.py:68: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  past_price = historical_data['Close'][0]
C:\Users\kchak\AppData\Local\Temp\ipykernel_9028\3222473881.py:68: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  past_price = hi

--> Processed 11/2081 stocks


...

--> Processed 2061/2081 stocks


C:\Users\kchak\AppData\Local\Temp\ipykernel_9028\3222473881.py:68: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  past_price = historical_data['Close'][0]
C:\Users\kchak\AppData\Local\Temp\ipykernel_9028\3222473881.py:59: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = current_price_data['Close'][-1]
C:\Users\kchak\AppData\Local\Temp\ipykernel_9028\3222473881.py:68: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  past_pri

--> Processed 2071/2081 stocks


C:\Users\kchak\AppData\Local\Temp\ipykernel_9028\3222473881.py:68: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  past_price = historical_data['Close'][0]
C:\Users\kchak\AppData\Local\Temp\ipykernel_9028\3222473881.py:59: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = current_price_data['Close'][-1]
C:\Users\kchak\AppData\Local\Temp\ipykernel_9028\3222473881.py:59: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_

--> Processed 2081/2081 stocks
✅ DataFrame built successfully.
✅ Results exported to 'nse_all_stocks_data_2025-04-27.csv' and 'nse_all_stocks_data_2025-04-27.xlsx' successfully.
✅ Results exported to 'nse_stocks_50_percent_rise_2025-04-27.csv' and 'nse_stocks_50_percent_rise_2025-04-27.xlsx' successfully.
✅ Results exported to 'nse_stocks_50_percent_drop_2025-04-27.csv' and 'nse_stocks_50_percent_drop_2025-04-27.xlsx' successfully.
# AI Agent execution completed successfully!
